In [697]:
import ast
def print_ast(src):
    print(ast.dump(src, indent=4))

In [698]:
def simple_function(x):
    y = 3 * x
    print(y)

In [699]:
import inspect

In [700]:
tree = ast.parse(inspect.getsource(simple_function))

In [ ]:
print_ast(tree)

In [ ]:
print_ast(tree.body[0])

In [701]:
import random

We create a PythonMutator class that links to other Mutator classes, the PythonMutator class tying all of them together with helper functions to call those class' methods. We can later use it to add probabilities etc. to each mutation.

In [ ]:
class PythonMutator:
    def visit_Module(self, src):
        return self.generic_visit(src)
    
    def visit_FunctionDef(self, src):
        return self.generic_visit(src)
    
    def visit_BinOp(self, src):
        return self.generic_visit(src)
    
    def visit_Assign(self, src):
        return self.generic_visit(src)
    
    def visit_Call(self, src):
        return self.generic_visit(src)
    
    def visit_Name(self, src):
        return self.generic_visit(src)
    
    def visit_Constant(self, src):
        return self.generic_visit(src)

    def expand_constants(self, src):
        return ExprMutator().modify_value(src)
    
    def swap_numbers(self, src):
        return ExprMutator().commute_value(src)

Now we need to define modify_value that can replace a given constant with an equivalent arithmetic expression, and swap_numbers that will swap the children of a + or * node.

In [702]:
class ExprMutator(ast.NodeTransformer):
    MAP = [("+", ast.Add()), ("*", ast.Mult()), ("/", ast.Div()), ("-", ast.Sub())]
    EXPAND = 1
    COMMUTE = 2

    def __init__(self):
        self.transform = False
        self.depth = 0
        self.mode = self.EXPAND

    def modify_value(self, n, depth=3):
        self.mode = self.EXPAND
        self.depth = depth
        return self._modify_value(n)
    
    def _modify_value(self, n):
        if self.depth == 0: return n
        self.transform = True
        return self._modify_value(self.visit(n))

We need the mode so we can swap between traversing a path and swapping children. Depth allows us to control how many numbers we want to go and replace with expressions.

In [703]:
class ExprMutator(ExprMutator):    
    def commute_value(self, n):
        self.mode = self.COMMUTE
        return self.visit(n)

Now come the real functions. The visits to Constant or BinOp nodes are what will truly handle the functionality.

In [734]:
class ExprMutator(ExprMutator):
    def visit_Constant(self, src):
        if isinstance(src.value, int) and self.transform and self.mode == self.EXPAND:
            while True:
                try:
                    op = random.randint(0, 3)
                    other = random.randint(-10000, 10000)
                    assert eval("(" + str(src.value) + self.MAP[3-op][0] + str(other) + ")" + self.MAP[op][0] + str(other)) == src.value
                    break
                except ZeroDivisionError: continue
                except AssertionError: continue
            self.depth -= 1
            self.transform = False
            return ast.BinOp(left = ast.Constant(value=eval("(" + str(src.value) + self.MAP[3-op][0] + str(other) + ")")), op = self.MAP[op][1], right = ast.Constant(value=other))
            
        return src

    def visit_BinOp(self, src):
        if self.mode == self.EXPAND:
            if random.randint(1, 2) == 1:
                src.left = self.visit(src.left)
            else:
                src.right = self.visit(src.right)
            return src
        
        if self.mode == self.COMMUTE:
            if isinstance(src.op, ast.Add) or isinstance(src.op, ast.Mult):
                src.left, src.right = src.right, src.left
                
            return self.generic_visit(src)

In [ ]:
print_ast(ExprMutator().modify_value(ast.Constant(value=1), depth=2))

In [786]:
ast.unparse(ast.fix_missing_locations(PythonMutator().expand_constants(ast.parse("x+1"))))

'x + 0.00042426813746287653 * (-5.193317422434368 * 1257 + 8885)'

In [ ]:
from copy import deepcopy

In [790]:
ast.unparse(ast.fix_missing_locations(PythonMutator().swap_numbers(ast.parse("x + 0.00042426813746287653 * (-5.193317422434368 * 1257 + 8885)"))))

'(8885 + 1257 * -5.193317422434368) * 0.00042426813746287653 + x'

In [792]:
tree_two = deepcopy(tree)
for i in range(5):
    if random.randint(1, 5) == 1: PythonMutator().swap_numbers(tree_two)
    else: PythonMutator().expand_constants(tree_two)
new_code = ast.unparse(tree_two)
print(new_code)

def simple_function(x):
    y = (-0.7600483529767301 * ((-22985682170 - 4597 + 9459) / (-1693 - 1666) / (0.2830550232685464 * -3653)) + -12.443069306930694 * (0.07876779099239618 * (-3461 + (14508 + -9182) - (-0.08678071539657854 * 9645 - 2427)))) * x
    print(y)


In [793]:
simple_function(456)

1368.0


In [794]:
exec(new_code)

In [795]:
simple_function(456)

1368.0


Clearly the output remains the same inspite of our changes. Next, we look into transforming range-based for loops into while loops.

In [ ]:
for_tree = ast.parse('''for i in range(10, 1, -2):
                        print(i)''')
print_ast(for_tree)

In [ ]:
while_tree = ast.parse('''
i = 10
while i > 1:
    print(i)
    i += -2''')
print_ast(while_tree)

In [ ]:
src = for_tree.body[0]
def analyze_for(node):
    args = node.iter.args
    if len(args) == 1:
        return [ast.Constant(value=0), ast.Lt(), args[0], ast.Constant(value=1)]
    elif len(args) == 2:
        return [args[0], ast.Lt(), args[1], ast.Constant(value=1)]
    else:
        step = eval(ast.unparse(args[2]))
        if step < 0:
            return [args[0], ast.Gt(), args[1], args[2]]
        else:
            return [args[0], ast.Lt(), args[1], args[2]]
        
while_args = analyze_for(src)
print_ast(
    ast.Assign(targets=[src.target], value=while_args[0])
    ) 
print_ast(
    ast.While(test=ast.Compare(left=ast.Name(id=src.target.id, ctx=ast.Load()), ops=[while_args[1]], comparators=[while_args[2]]), \
              body=src.body + [ast.AugAssign(target=src.target, op=ast.Add(), value=while_args[3])], orelse=src.orelse)
)

In [ ]:
class PythonMutator(PythonMutator):
    def transform_for(self, src):
        return ForMutator().visit(src)

In [ ]:
class ForMutator(ast.NodeTransformer):
    def visit_For(self, src):  
        if isinstance(src.iter, ast.Name): return src
        while_args = analyze_for(src)

        return [ast.Assign(targets=[src.target], value=while_args[0]), \
                ast.While(test=ast.Compare(left=ast.Name(id=src.target.id, ctx=ast.Load()), ops=[while_args[1]], comparators=[while_args[2]]), \
                          body=src.body + [ast.AugAssign(target=src.target, op=ast.Add(), value=while_args[3])], orelse=src.orelse)]

In [705]:
for_tree_two = deepcopy(for_tree)
print(ast.unparse(for_tree))
print("====")
print(ast.unparse(ast.fix_missing_locations(PythonMutator().transform_for(for_tree_two))))

for i in range(10, 1, -2):
    print(i)
====
i = 10
while i > 1:
    print(i)
    i += -2


That takes care of for-loops based on ranges. <b>What about iterators?</b>

In [ ]:
print_ast(ast.parse(
'''
L = [1, 4, "hello"]
for i in [len(str(x)) for x in L]:
    print(i)
'''
))